In [1]:
NUM_OF_CLASSES = 4  # 1 for temp.pth because we disregarded classification to pedestrians, people in vehicles etc.
                    # 4 for trained_model_old.pth beacuse there it wasn't disregarded
FILE_NAME = 'trained_model_old.pth'  # 'temp.pth' or 'trained_model_old.pth' or anthing available
TEST_IMAGE_PATH = 'data/HT21/test/HT21-15/img1/000007.jpg'

In [2]:
import torch
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw

# Define a function to visualize the model's output
def visualize_output(image, output):
    image = T.ToPILImage()(image)  # Convert the image to PIL format
    draw = ImageDraw.Draw(image)
    
    boxes = output['boxes'].detach().cpu().numpy()
    labels = output['labels'].detach().cpu().numpy()
    
    for box, label in zip(boxes, labels):
        draw.rectangle(box, outline='red', width=3)
        draw.text((box[0], box[1]), f"Class: {label}", fill='red')
    
    image.show()
    
# Load the trained model
model = fasterrcnn_resnet50_fpn(num_classes=NUM_OF_CLASSES)  # Adjust num_classes based on your dataset
model.load_state_dict(torch.load(FILE_NAME))
model.eval()

# Open image
test_image_path = TEST_IMAGE_PATH
test_image = Image.open(test_image_path).convert("RGB")
test_image_tensor = T.ToTensor()(test_image).unsqueeze(0)

# Run the model on the test image
with torch.no_grad():
    output = model(test_image_tensor)

# Visualize the output
visualize_output(test_image_tensor.squeeze(0), output[0])